## ROUTE 3 OUTBOUND 6 PM - 6 AM PEAK

In [1]:
import geopandas as gpd 
import requests 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import json
from shapely.geometry import LineString
import folium

%matplotlib inline

In [2]:
with open('../data/WeGo.json')as start:
         credentials = json.load(start)

In [3]:
Authorization = credentials['Authorization']

In [4]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/3'

In [5]:
def concat_frames_columns(geo_response, response, date_range, time_range='6am-9am', days='m-f', in_or_out='inbound'):
    """This function pulls in two API requests, puts one in a geodataframe the other in a dataframe and concats desired columns"""
    geo_df = gpd.GeoDataFrame.from_features(geo_response.json()['features'])
    res_df = pd.DataFrame(response.json()['data']['segments'])
    full_df = pd.concat([geo_df, res_df[['countForSegment', 'percent20Secs']]], axis = 1)
    full_df['dates'] = date_range
    full_df['time'] = time_range
    full_df['days_of_week'] = days
    full_df['direction']= in_or_out
    full_df['max_seg_count'] = full_df['countForSegment'].max()
    full_df['proportion_of_segments'] = full_df['countForSegment']/full_df['max_seg_count']
    return full_df

## In School Months - Jan to May

In [6]:
params1a = {
    'direction':'0',
    'startDate':'01-25-2022',
    'endDate': '02-24-2022',
    'excludeDates': '02-18-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params1b = {
    'direction':'0',
    'startDate':'01-25-2022',
    'endDate': '02-24-2022',
    'excludeDates': '02-18-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [7]:
mfam_jantofeb = concat_frames_columns(requests.get(URL, params = params1a, headers = headers), 
                                      requests.get(URL, params = params1b, headers = headers), "4/27/22-5/25/22")
mfam_jantofeb

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,3.409089,7.124484,19.741,5.283,...,1.954,1.612,135,2.090,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.525292
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,4.115733,5.885464,45.026,8.395,...,1.662,1.339,176,1.760,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.684825
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,2.156142,11.533563,20.780,2.825,...,1.475,1.204,176,1.552,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.684825
3,"LINESTRING (-86.81680 36.14151, -86.81698 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.781971,8.694555,16.869,4.765,...,1.690,1.476,173,1.834,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.673152
4,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.846557,13.185079,4.416,2.585,...,1.408,1.118,183,1.499,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.712062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,"LINESTRING (-86.85766 36.13797, -86.85761 36.1...",WHIKNONN,White Bridge Pike & Knob Rd Nb,WHIKENNN,White Bridge Pike & Kendall Dr Nb,24.392,2.693534,9.055761,9.906,4.735,...,1.513,1.313,148,1.666,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.575875
510,"LINESTRING (-86.81570 36.14246, -86.81588 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.651069,9.123867,5.272,3.696,...,1.873,1.476,173,2.022,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.673152
511,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,2.039120,12.114049,15.228,2.564,...,1.503,1.250,183,1.560,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.712062
512,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.705459,9.103076,10.112,3.844,...,1.816,1.419,181,2.036,4/27/22-5/25/22,6am-9am,m-f,inbound,257,0.704280


In [8]:
r3feb = mfam_jantofeb[mfam_jantofeb['percent20Secs'] > np.quantile(mfam_jantofeb['percent20Secs'], .80)]

In [9]:
r3feb_gj = r3feb['geometry'].to_json()

In [10]:
mapdawg = folium.Map(location = [36.19, -86.7775], zoom_start = 12)
folium.GeoJson(r3feb_gj).add_to(mapdawg)
mapdawg

In [11]:
params2a = {
    'direction':'0',
    'startDate':'02-25-2022',
    'endDate': '03-26-2022',
    'excludeDates': '03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params2b = {
    'direction':'0',
    'startDate':'02-25-2022',
    'endDate': '03-26-2022',
    'excludeDates': '03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [12]:
mfam_febtomar = concat_frames_columns(requests.get(URL, params = params2a, headers = headers), 
                                      requests.get(URL, params = params2b, headers = headers), "4/27/22-5/25/22")
mfam_febtomar

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.724030,8.916201,7.450,3.793,...,1.830,1.477,101,2.007,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.726619
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,7.463646,3.245465,65.835,24.254,...,1.731,1.428,130,1.799,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.935252
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,2.131977,11.664291,11.673,3.139,...,1.462,1.272,130,1.525,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.935252
3,"LINESTRING (-86.81657 36.14171, -86.81675 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.712578,8.916979,5.799,4.405,...,1.788,1.535,90,1.923,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.647482
4,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.841435,13.221754,4.739,2.665,...,1.393,1.153,138,1.472,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.992806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,"LINESTRING (-86.81377 36.14412, -86.81396 36.1...",WES27AWN,West End Ave & 27th Ave S Wb,WES31AWN,West End Ave & 31st Ave N Wb,24.810,2.423025,10.239267,4.715,3.714,...,1.815,1.639,40,1.904,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.287770
540,"LINESTRING (-86.81547 36.14266, -86.81565 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.591178,9.334750,4.971,4.059,...,1.880,1.601,90,1.971,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.647482
541,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,2.198130,11.237734,24.914,3.360,...,1.445,0.580,138,1.559,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.992806
542,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.927793,8.411797,12.992,4.668,...,1.865,1.545,135,2.037,4/27/22-5/25/22,6am-9am,m-f,inbound,139,0.971223


In [13]:
r3march = mfam_febtomar[mfam_febtomar['percent20Secs'] > np.quantile(mfam_febtomar['percent20Secs'], .80)]

In [14]:
r3march_gj =  r3march['geometry'].to_json()

In [15]:
mapdawg1 = folium.Map(location = [36.19, -86.7775], zoom_start = 13)
folium.GeoJson(r3march_gj).add_to(mapdawg1)
mapdawg1

In [16]:
params3a = {
    'direction':'0',
    'startDate':'03-27-2022',
    'endDate': '04-26-2022',
    'excludeDates': '04-14-2022,04-15-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params3b = {
    'direction':'0',
    'startDate':'03-27-2022',
    'endDate': '04-26-2022',
    'excludeDates': '04-14-2022,04-15-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [17]:
mfam_martoapr = concat_frames_columns(requests.get(URL, params = params3a, headers = headers), 
                                      requests.get(URL, params = params3b, headers = headers), "4/27/22-5/25/22")
mfam_martoapr

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.86187 36.11438, -86.86204 36.1...",HARDAVWF,Harding Pike & Davidson Rd Wb,HARPOSSN,Harding Pike & Post Rd Sb,24.760,1.707409,14.501505,3.337,2.242,...,1.285,1.118,137,1.378,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.452145
1,"LINESTRING (-86.94569 36.07581, -86.94561 36.0...",SAWPLANM,Sawyer Brown Rd & Plantation Dr Nb,HWYSAWWM,Hwy 70 S & Sawyer Brown Wb,23.973,2.704313,8.864728,38.461,3.767,...,1.622,0.969,150,1.725,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.495050
2,"LINESTRING (-86.93375 36.06927, -86.93399 36.0...",COLBAUSN,Colice Jeanne Rd & Baugh Rd Sb,BAUSHEWN,Baugh Rd & Sheffield Sq Wb,22.970,5.008090,4.586579,200.040,5.432,...,1.967,1.059,144,2.199,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.475248
3,"LINESTRING (-86.92813 36.07392, -86.92840 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,1.447664,17.039866,2.200,1.743,...,1.206,1.003,149,1.274,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.491749
4,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,5.089609,4.759305,65.489,9.275,...,1.657,0.971,297,1.740,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.980198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,"LINESTRING (-86.95485 36.07860, -86.95511 36.0...",H70BAPWM,Hwy 70 S & Baptist Bellevue Medical Wb,COLDAVPR,Coley Davis Rd Shelter Park-N-Ride,24.143,3.650239,6.614087,175.411,2.789,...,1.881,1.656,142,1.956,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.468647
1098,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.996445,12.372993,21.637,2.269,...,1.410,1.269,173,1.500,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.570957
1099,"LINESTRING (-86.93470 36.07396, -86.93497 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,2.992599,8.243002,31.200,4.245,...,1.663,1.029,147,1.824,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.485149
1100,"LINESTRING (-86.87423 36.09808, -86.87447 36.0...",HARPAGSN,Harding Pike & Page Rd Sb,H70VOSWN,Highway 70 S & Vossland Dr Wb,24.785,1.522511,16.279029,2.628,1.786,...,1.295,0.813,141,1.344,4/27/22-5/25/22,6am-9am,m-f,inbound,303,0.465347


In [18]:
r3apr = mfam_martoapr[mfam_martoapr['percent20Secs'] > np.quantile(mfam_martoapr['percent20Secs'], .80)]

In [19]:
r3apr.describe()

,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,percent80Secs,percent70Secs,medianSecs,percent30Secs,percent10Secs,minSecs,countForSegment,percent20Secs,max_seg_count,proportion_of_segments
count,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.0,221.000000
mean,24.358982,9.305378,5.735781,94.221991,22.675760,14.994778,7.420946,4.139787,3.012416,2.161086,1.460643,223.909502,2.673719,303.0,0.738975
std,0.553813,34.386712,2.738681,274.718986,120.358287,84.125152,20.839053,4.549441,1.257441,0.826338,0.559544,76.227834,0.846084,0.0,0.251577
min,22.139000,2.195107,0.063328,4.253000,2.571000,2.397000,2.294000,2.170000,2.020000,0.568000,0.455000,10.000000,1.945000,303.0,0.033003
25%,24.143000,3.045378,3.712663,11.866000,4.109000,3.357000,3.030000,2.559000,2.170000,1.839000,1.213000,150.000000,2.028000,303.0,0.495050
50%,24.465000,4.490394,5.439160,38.461000,6.589000,4.821000,4.036000,3.117000,2.653000,1.955000,1.501000,282.000000,2.391000,303.0,0.930693
75%,24.805000,6.679135,8.097058,78.694000,10.951000,6.899000,5.728000,4.325500,3.469000,2.596000,1.691000,288.000000,3.073000,303.0,0.950495
max,24.984000,381.239583,11.344777,2719.033000,1351.238000,1114.119000,280.566000,59.220000,12.563000,4.786000,4.089000,301.000000,6.839000,303.0,0.993399


In [20]:
r3apr_gj = r3apr['geometry'].to_json()

In [21]:
mapdawg2 = folium.Map(location = [36.19, -86.7775], zoom_start = 13)
folium.GeoJson(r3apr_gj).add_to(mapdawg2)
mapdawg2

In [22]:
params4a = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '18:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '23:59',
    'format' : 'geojson',
    'resolution': 'hiRes',
}


params4b = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '18:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '23:59',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [23]:
mfam_aprtomay = concat_frames_columns(requests.get(URL, params = params4a, headers = headers), 
                                      requests.get(URL, params = params4b, headers = headers), "4/27/22-5/25/22")
mfam_aprtomay

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.86187 36.11438, -86.86204 36.1...",HARDAVWF,Harding Pike & Davidson Rd Wb,HARPOSSN,Harding Pike & Post Rd Sb,24.760,1.708943,14.488488,5.426,2.312,...,1.238,0.900,193,1.342,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.536111
1,"LINESTRING (-86.94569 36.07581, -86.94561 36.0...",SAWPLANM,Sawyer Brown Rd & Plantation Dr Nb,HWYSAWWM,Hwy 70 S & Sawyer Brown Wb,23.973,2.470023,9.705578,20.286,3.583,...,1.552,0.997,214,1.660,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.594444
2,"LINESTRING (-86.93375 36.06927, -86.93399 36.0...",COLBAUSN,Colice Jeanne Rd & Baugh Rd Sb,BAUSHEWN,Baugh Rd & Sheffield Sq Wb,22.970,7.895042,2.909421,368.179,6.837,...,2.006,0.778,213,2.192,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.591667
3,"LINESTRING (-86.92813 36.07392, -86.92840 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,1.432164,17.224284,2.199,1.727,...,1.214,1.097,207,1.275,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.575000
4,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,5.350404,4.527322,76.953,11.627,...,1.701,1.372,354,1.811,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.983333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,"LINESTRING (-86.95485 36.07860, -86.95511 36.0...",H70BAPWM,Hwy 70 S & Baptist Bellevue Medical Wb,COLDAVPR,Coley Davis Rd Shelter Park-N-Ride,24.143,2.230910,10.822041,7.787,2.572,...,1.880,1.445,210,1.967,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.583333
1098,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,2.056405,12.012225,29.660,2.550,...,1.410,0.767,185,1.491,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.513889
1099,"LINESTRING (-86.93470 36.07396, -86.93497 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,3.330129,7.407521,30.022,6.039,...,1.573,0.760,210,1.810,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.583333
1100,"LINESTRING (-86.85804 36.14839, -86.85830 36.1...",WHIBURNN,White Bridge Pike & Burgess Ave Nb,CXIWTBRG,White Bridge Station Inbound,24.360,2.627536,9.271043,11.612,4.025,...,0.591,0.459,153,1.165,4/27/22-5/25/22,6am-9am,m-f,inbound,360,0.425000


In [24]:
r3may = mfam_aprtomay[mfam_aprtomay['percent20Secs'] > np.quantile(mfam_aprtomay['percent20Secs'], .80)]

In [25]:
r3may_gj = r3may['geometry'].to_json()

In [26]:
mapdawg3 = folium.Map(location = [36.19, -86.7775], zoom_start = 13)
folium.GeoJson(r3may_gj).add_to(mapdawg3)
mapdawg3

## Summer Months

In [27]:
params5a = {
    'direction':'0',
    'startDate':'05-26-2022',
    'endDate': '06-25-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params5b = {
    'direction':'0',
    'startDate':'05-26-2022',
    'endDate': '06-25-2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [28]:
mfam_maytojun = concat_frames_columns(requests.get(URL, params = params5a, headers = headers), 
                                      requests.get(URL, params = params5b, headers = headers), "4/27/22-5/25/22")
mfam_maytojun

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.86187 36.11438, -86.86204 36.1...",HARDAVWF,Harding Pike & Davidson Rd Wb,HARPOSSN,Harding Pike & Post Rd Sb,24.760,1.614404,15.336929,6.969,2.180,...,1.227,0.690,198,1.331,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.521053
1,"LINESTRING (-86.94569 36.07581, -86.94561 36.0...",SAWPLANM,Sawyer Brown Rd & Plantation Dr Nb,HWYSAWWM,Hwy 70 S & Sawyer Brown Wb,23.973,2.328027,10.297561,10.401,3.366,...,1.517,1.007,221,1.631,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.581579
2,"LINESTRING (-86.93375 36.06927, -86.93399 36.0...",COLBAUSN,Colice Jeanne Rd & Baugh Rd Sb,BAUSHEWN,Baugh Rd & Sheffield Sq Wb,22.970,10.576977,2.171698,242.991,10.762,...,2.111,0.699,220,2.326,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.578947
3,"LINESTRING (-86.92813 36.07392, -86.92840 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,1.433018,17.214020,3.069,1.709,...,1.219,0.922,218,1.258,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.573684
4,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,8.581694,2.822636,87.986,22.853,...,1.670,1.350,372,1.809,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.978947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,"LINESTRING (-86.95485 36.07860, -86.95511 36.0...",H70BAPWM,Hwy 70 S & Baptist Bellevue Medical Wb,COLDAVPR,Coley Davis Rd Shelter Park-N-Ride,24.143,2.199242,10.977873,9.587,2.478,...,1.874,1.202,211,1.966,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.555263
1098,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.979132,12.481229,15.354,2.906,...,1.348,1.191,190,1.409,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.500000
1099,"LINESTRING (-86.93470 36.07396, -86.93497 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,3.006706,8.204327,26.076,4.663,...,1.572,0.715,221,1.774,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.581579
1100,"LINESTRING (-86.85804 36.14839, -86.85830 36.1...",WHIBURNN,White Bridge Pike & Burgess Ave Nb,CXIWTBRG,White Bridge Station Inbound,24.360,2.465834,9.879011,6.053,3.938,...,0.694,0.611,169,1.142,4/27/22-5/25/22,6am-9am,m-f,inbound,380,0.444737


In [29]:
r3june = mfam_maytojun[mfam_maytojun['percent20Secs'] > np.quantile(mfam_maytojun['percent20Secs'], .80)]

In [30]:
r3june_gj = r3june['geometry'].to_json()

In [31]:
mapdawg4 = folium.Map(location = [36.19, -86.7775], zoom_start = 12)
folium.GeoJson(r3june_gj).add_to(mapdawg4)
mapdawg4

In [32]:
params6a = {
    'direction':'0',
    'startDate':'06-26-2022',
    'endDate': '07-26-2022',
    'excludeDate': '07/04/2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params6b = {
    'direction':'0',
    'startDate':'06-26-2022',
    'endDate': '07-26-2022',
    'excludeDate': '07/04/2022',
    'beginTime': '18:00',
    'endTime': '23:59',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [33]:
mfam_juntojul = concat_frames_columns(requests.get(URL, params = params6a, headers = headers), 
                                      requests.get(URL, params = params6b, headers = headers), "4/27/22-5/25/22")
mfam_juntojul

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.86187 36.11438, -86.86204 36.1...",HARDAVWF,Harding Pike & Davidson Rd Wb,HARPOSSN,Harding Pike & Post Rd Sb,24.760,1.603060,15.445461,4.746,2.112,...,1.198,0.960,199,1.335,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.519582
1,"LINESTRING (-86.94569 36.07581, -86.94561 36.0...",SAWPLANM,Sawyer Brown Rd & Plantation Dr Nb,HWYSAWWM,Hwy 70 S & Sawyer Brown Wb,23.973,2.330380,10.287163,14.498,3.428,...,1.535,0.778,216,1.657,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.563969
2,"LINESTRING (-86.93375 36.06927, -86.93399 36.0...",COLBAUSN,Colice Jeanne Rd & Baugh Rd Sb,BAUSHEWN,Baugh Rd & Sheffield Sq Wb,22.970,9.537549,2.408376,194.314,12.417,...,2.067,0.893,213,2.276,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.556136
3,"LINESTRING (-86.92813 36.07392, -86.92840 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,1.469825,16.782950,3.408,1.743,...,1.188,0.866,217,1.259,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.566580
4,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,6.839070,3.541856,65.080,19.003,...,1.623,1.372,374,1.748,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.976501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,"LINESTRING (-86.95485 36.07860, -86.95511 36.0...",H70BAPWM,Hwy 70 S & Baptist Bellevue Medical Wb,COLDAVPR,Coley Davis Rd Shelter Park-N-Ride,24.143,2.337917,10.326714,8.167,2.681,...,1.907,1.720,206,1.983,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.537859
1098,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,2.436068,10.140111,25.963,3.694,...,1.371,1.214,190,1.466,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.496084
1099,"LINESTRING (-86.93470 36.07396, -86.93497 36.0...",H70HICWN,Hwy 70 S & Hicks Rd Wb,COLHWYSF,Colice Jeanne Rd & Hwy 70 Sb,24.668,3.115405,7.918072,11.199,5.635,...,1.559,0.829,215,1.749,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.561358
1100,"LINESTRING (-86.87423 36.09808, -86.87447 36.0...",HARPAGSN,Harding Pike & Page Rd Sb,H70VOSWN,Highway 70 S & Vossland Dr Wb,24.785,1.438596,17.228603,2.081,1.681,...,1.253,1.074,203,1.302,4/27/22-5/25/22,6am-9am,m-f,inbound,383,0.530026


In [34]:
r3july = mfam_juntojul[mfam_juntojul['percent20Secs'] > np.quantile(mfam_juntojul['percent20Secs'], .80)]

In [35]:
r3july_gj = r3july['geometry'].to_json()

In [36]:
mapdawg5 = folium.Map(location = [36.19, -86.7775], zoom_start = 12)
folium.GeoJson(r3july_gj).add_to(mapdawg5)
mapdawg5

In [37]:
school = pd.concat([r3feb, r3march, r3apr, r3may], keys=['r3feb', 'r3march', 'r3apr', 'r3may'])
summer = pd.concat([r3june, r3july], keys=['r3june', 'r3july'])
r3oischool = school['geometry'].to_json()
r3oisummer = summer['geometry'].to_json()

In [38]:
mapdawg6 = folium.Map(location = [36.19, -86.7775], zoom_start = 12)

In [39]:
style1 = {'color' : 'red', 'weight' : 6}
style2 = {'color' : 'blue', 'weight' : 3}
folium.GeoJson(r3oischool, style_function =lambda x:style1).add_to(mapdawg6)
folium.GeoJson(r3oisummer, style_function = lambda x:style2).add_to(mapdawg6)
mapdawg6